# Data Analysis Test Playground

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import os
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import glob

pd.set_option('display.max_rows', None)

# This requires Python 3.7 and above
print('Pandas version:', pd.__version__)
print('NumPy version:', np.__version__)



Pandas version: 2.2.3
NumPy version: 2.0.2


## File Helping Functions

### CSV File Reader Function

Reads multiple CSV files starting with "FED" from a directory and combines them into a single DataFrame.

#### What it does
- Scans directory for "FED*.CSV" files
- Reads each file and adds a `file_num` identifier column
- Combines all files into one DataFrame
- Converts "MM:DD:YYYY hh:mm:ss" to datetime format

#### Usage
```python
# Import and use
from your_module import read_csv_files
combined_data = read_csv_files("/path/to/csv/files")

# Work with results
print(f"Total records: {len(combined_data)}")
```

#### Notes
- Handles file reading errors gracefully
- Returns empty DataFrame if no valid files found
- Datetime sorting is available (commented out by default)

In [3]:
def read_csv_files(file_path):
    """
    Reads CSV files that start with 'FED' and organizes them into a single dataframe with file categories.
    
    Args:
        file_path (str): Path to the directory containing CSV files
        
    Returns:
        pandas.DataFrame: A single dataframe containing data from all FED CSV files
    """
    all_dataframes = []
    file_counter = 1  # Initialize counter for file categories

    # For all files in folder
    for file in os.listdir(file_path):
        if file.endswith(".CSV") and file.startswith("FED"):
            # Read that file into a dataframe
            file_path_df = os.path.join(file_path, file)
            try:
                df = pd.read_csv(file_path_df)
                
                # Add file category column
                df["file_num"] = file_counter
                file_counter += 1
                
                all_dataframes.append(df)
            except Exception as e:
                print(f"Error reading file {file}: {e}")
    
    if not all_dataframes:
        print("No valid FED CSV files found in the specified directory.")
        return pd.DataFrame()
        
    # Create a single dataframe from all files
    singular_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Convert datetime column to datetime type
    try:
        singular_df["MM:DD:YYYY hh:mm:ss"] = pd.to_datetime(singular_df["MM:DD:YYYY hh:mm:ss"])
        # Uncomment the line below to sort by datetime
        # singular_df = singular_df.sort_values(by=['MM:DD:YYYY hh:mm:ss'], ascending=True)
    except KeyError:
        print("Warning: 'MM:DD:YYYY hh:mm:ss' column not found. Skipping datetime conversion.")
    except Exception as e:
        print(f"Error converting datetime: {e}")
    
    return singular_df

## Lab Data File Path Setup and Directory File Count

#### This code defines file path variables for a data analysis project in the Fobbs Lab. 
- It specifies paths to directories containing mouse experimental data, particularly for the "Chow Group" and a specific mouse (M281) with a feeding device (FED004). 
- It also defines variables for the mouse ID, feeding device ID, and date (February 23, 2025). 
- Finally, it prints the number of files found in the individual mouse's directory by using the os.listdir() function to count files and displaying the result.

In [3]:

MICE_GROUP_PATH ='/Users/kevinmcpherson/github-projects/fobbs-lab/data-analysis/local_files/input/Chow Group/'
INDIVIDUAL_MOUSE_PATH = '/Users/kevinmcpherson/github-projects/fobbs-lab/data-analysis/local_files/input/Chow Group/m281_FED004/'
SD_ANALYSIS_FILES_PATH = '/Users/kevinmcpherson/github-projects/fobbs-lab/data-analysis/local_files/input/SD Analyses/'
MOUSE = 'M281'
FED = '_FED004'
DATE = '022325'


print ("Number of files found: ")
print(len(os.listdir(INDIVIDUAL_MOUSE_PATH)))

Number of files found: 
39


## Processing FED Device Data Files

This script processes FED (Feeding Experimentation Device) data files by adding two new columns that track the beginning and end dates for each file number in the dataset.

### Input Data Structure
The script expects a CSV file with the following key columns:
- An unnamed index column (first column)
- `MM:DD:YYYY hh:mm:ss`: Timestamp column in the format "M/D/YYYY HH:MM:SS"
- `file_num`: Integer column indicating the file number

### New Columns Added
The script adds two new columns to the dataset:
- `file_begin_date`: The date (MM/DD/YYYY) when each file_num first appears
- `file_end_date`: The date (MM/DD/YYYY) when each file_num last appears

### Processing Steps
1. Reads the CSV file using pandas, setting the unnamed first column as the index
2. Converts the "MM:DD:YYYY hh:mm:ss" column to datetime format for proper date handling
3. Groups the data by file_num to identify:
   - First timestamp for each file number (begin_date)
   - Last timestamp for each file number (end_date)
4. Creates new columns mapping these dates back to the original dataframe
5. Formats dates in MM/DD/YYYY format
6. Saves the processed data to a new CSV file

### Usage
```python
input_file = "SD Analyses M281 FED004.csv"
output_file = "SD Analyses M281 FED004_processed.csv"
processed_df = process_fed_dates(input_file, output_file)
```

### Output
The script creates a new CSV file with all original columns plus:
- file_begin_date
- file_end_date

It also prints:
- A sample of the processed data (first 5 rows)
- A summary showing the date ranges for each file number

Note: The original timestamp format is preserved in the "MM:DD:YYYY hh:mm:ss" column, while the new date columns are formatted as MM/DD/YYYY for clarity.

In [5]:
def process_fed_dates(input_file, output_file):
    """
    Process FED device CSV file to add file_begin_date and file_end_date columns.

    Parameters:
    input_file (str): Path to input CSV file
    output_file (str): Path to save the processed CSV file
    """
    # Read the CSV file
    # Note: the first column is unnamed and just an index, so we'll use it as the index
    df = pd.read_csv(input_file, index_col=0)

    # Convert datetime column to datetime type
    df['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'])

    # Group by file_num and get first and last dates
    file_dates = df.groupby('file_num').agg({
        'MM:DD:YYYY hh:mm:ss': ['first', 'last']
    })

    # Rename columns for clarity
    file_dates.columns = ['begin_date', 'end_date']

    # Convert to dictionary for mapping
    begin_dates = file_dates['begin_date'].to_dict()
    end_dates = file_dates['end_date'].to_dict()

    # Add new columns to original dataframe
    df['file_begin_date'] = df['file_num'].map(begin_dates).dt.strftime('%m/%d/%Y')
    df['file_end_date'] = df['file_num'].map(end_dates).dt.strftime('%m/%d/%Y')

    # Save to new CSV file
    df.to_csv(output_file)

    return df

## CSV Reader Function Explanation

### This function processes multiple CSV files from a specified directory and combines them into a single pandas DataFrame. Here's what it does:

1. **Filters Files**: It only processes files that:
   - Have a `.CSV` extension
   - Start with `FED` in their filename

2. **Adds Metadata**: Each file's data is tagged with a sequential number (`file_num`) to track which file the data came from.

3. **Combines Data**: All individual DataFrames are concatenated into a single DataFrame.

4. **Date Processing**: It converts a date-time column named `MM:DD:YYYY hh:mm:ss` to pandas datetime format for better date handling.

5. **Optional Sorting**: Contains a commented-out line that would sort the data by the date-time column if uncommented.

This function is useful for analyzing data spread across multiple CSV files that follow a similar format, particularly for time series data from multiple collection periods.

In [6]:
def read_csv_files(file_path):
    """
    Reads CSV files that start with 'FED' from a directory and combines them 
    into a single dataframe with file numbering.
    
    Args:
        file_path (str): Path to the directory containing CSV files
        
    Returns:
        pandas.DataFrame: Combined dataframe with all CSV data and file numbers
    """
    all_dataframes = []
    file_counter = 1
    
    # Iterate through all files in the specified directory
    for file in os.listdir(file_path):
        # Process only CSV files that start with "FED"
        if file.endswith(".CSV") and file.startswith("FED"):
            # Construct the full file path
            file_path_df = os.path.join(file_path, file)
            
            # Read the CSV into a dataframe
            df = pd.read_csv(file_path_df)
            
            # Add a file number identifier column
            df['file_num'] = file_counter
            file_counter += 1
            
            # Add to our collection
            all_dataframes.append(df)
    
    # Exit early if no files were found
    if not all_dataframes:
        return pd.DataFrame()
        
    # Combine all dataframes into one
    singular_df = pd.concat(all_dataframes)
    
    # Convert date-time column to datetime type
    singular_df['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(singular_df['MM:DD:YYYY hh:mm:ss'])
    
    # Uncomment the below line to sort by date-time
    # singular_df = singular_df.sort_values(by=['MM:DD:YYYY hh:mm:ss'], ascending=True)
    
    return singular_df

### Usage

#### Make concatentated file for `input_file` field

In [7]:
# Read and concatenate all FED CSV files from the individual mouse directory
# INDIVIDUAL_MOUSE_PATH should contain the specific FED folder for the mouse being analyzed
concatenated_dataframe = read_csv_files(INDIVIDUAL_MOUSE_PATH)

# Define the specific output path for saving the concatenated data
# This will save the file in the specified directory with a formatted filename
output_path = "/Users/kevinmcpherson/github-projects/fobbs-lab/data-analysis/local_files/output"

# Construct the full output filename with mouse ID, FED device number, and date
output_filename = f"{MOUSE}_{FED}_concat1_{DATE}.csv"

# Combine the path and filename for the complete file location
output_file_path = os.path.join(output_path, output_filename)

# Save the concatenated data to the specified file path
concatenated_dataframe.to_csv(output_file_path)

# Display the first 10 rows of the concatenated dataframe for verification
concatenated_dataframe.head(10)

,MM:DD:YYYY hh:mm:ss,Library_Version,Session_type,Device_Number,Battery_Voltage,Motor_Turns,FR,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time,file_num
0,2024-04-10 12:44:25,1.14.0,ClosedEcon,4,3.83,NaN,1,Left,Left,1,0,0,0,NaN,NaN,0.22,1
1,2024-04-10 12:44:33,1.14.0,ClosedEcon,4,3.83,3.0,1,Pellet,Left,1,0,1,1,1.89,NaN,NaN,1
2,2024-04-10 12:45:53,1.14.0,ClosedEcon,4,3.83,NaN,2,Left,Left,2,0,1,1,NaN,NaN,0.17,1
3,2024-04-10 12:45:57,1.14.0,ClosedEcon,4,3.83,NaN,2,Left,Left,3,0,1,1,NaN,NaN,0.05,1
4,2024-04-10 12:46:00,1.14.0,ClosedEcon,4,3.83,1.0,2,Pellet,Left,3,0,2,2,1.53,88.0,NaN,1
5,2024-04-10 12:47:24,1.14.0,ClosedEcon,4,3.83,NaN,3,Left,Left,4,0,2,2,NaN,NaN,0.11,1
6,2024-04-10 12:50:32,1.14.0,ClosedEcon,4,3.83,NaN,3,Left,Left,5,0,2,2,NaN,NaN,0.21,1
7,2024-04-10 12:50:34,1.14.0,ClosedEcon,4,3.83,NaN,3,Left,Left,6,0,2,2,NaN,NaN,0.13,1
8,2024-04-10 12:50:35,1.14.0,ClosedEcon,4,3.83,1.0,3,Pellet,Left,6,0,3,3,0.42,275.0,NaN,1
9,2024-04-10 12:54:14,1.14.0,ClosedEcon,4,3.83,NaN,4,Right,Left,6,1,3,3,NaN,NaN,0.05,1


In [8]:
if __name__ == "__main__":
    input_file = '/Users/kevinmcpherson/github-projects/fobbs-lab/data-analysis/local_files/output/M281__FED004_concat1_022325.csv'
    output_file = '/Users/kevinmcpherson/github-projects/fobbs-lab/data-analysis/local_files/output/M281__FED004_concat2_022325.csv'

    processed_df = process_fed_dates(input_file, output_file)
    print("Processing complete. New file saved as:", output_file)

    # Display sample of processed data
    print("\nSample of processed data (first 5 rows):")
    print(processed_df[['file_num', 'MM:DD:YYYY hh:mm:ss', 'file_begin_date', 'file_end_date']].head())

    # Print summary of file numbers and their date ranges
    summary = processed_df.groupby('file_num').agg({
        'file_begin_date': 'first',
        'file_end_date': 'first'
    })
    print("\nSummary of file number date ranges:")
    print(summary.head())

Processing complete. New file saved as: /Users/kevinmcpherson/github-projects/fobbs-lab/data-analysis/local_files/output/M281__FED004_concat2_022325.csv

Sample of processed data (first 5 rows):
   file_num MM:DD:YYYY hh:mm:ss file_begin_date file_end_date
0         1 2024-04-10 12:44:25      04/10/2024    04/11/2024
1         1 2024-04-10 12:44:33      04/10/2024    04/11/2024
2         1 2024-04-10 12:45:53      04/10/2024    04/11/2024
3         1 2024-04-10 12:45:57      04/10/2024    04/11/2024
4         1 2024-04-10 12:46:00      04/10/2024    04/11/2024

Summary of file number date ranges:
         file_begin_date file_end_date
file_num                              
1             04/10/2024    04/11/2024
2             04/11/2024    04/12/2024
3             03/28/2024    03/29/2024
4             03/29/2024    03/30/2024
5             04/24/2024    04/25/2024
